In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
from pathlib import Path
import seaborn as sns
import numpy as np
import glob
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, accuracy_score, auc, precision_recall_fscore_support, pairwise, f1_score, log_loss, make_scorer
from sklearn.metrics import precision_score, recall_score
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.utils import validation
from scipy.sparse import issparse
from scipy.spatial import distance
from sklearn.calibration import CalibratedClassifierCV
import pickle


RANDOM_STATE = 15485867

%matplotlib inline
plt.style.use('seaborn-white')


%load_ext autotime

In [ ]:
import sklearn
sklearn.show_versions()

In [ ]:
from modeling_fxn import evaluate, hypertuning_fxn, hypertuned_cv_fxn, classifier_eval, plot_roc, optimal_youden_index, saveplot

In [ ]:
################# general configs #################
## savepath configs: ##
# date= '30_06_2022'
# folder='24_hr_window'

## crossvalidation and modeling configs ##
nfolds=10
scoring='roc_auc' #neg_log_loss
n_iter=40 #for gridsearch
gridsearch=False #gridsearch=False means it does triaged hyperparameter combinations based on some algorithm. True= tests all 

## refpath configs ##
repo_path=os.getcwd() #replace with repository path.
new_data_path= os.path.join(repo_path, 'data' )

## saving bool for plot functions ##
save_boolean=False

In [ ]:
############## data import ##############
###all NM-T + NM-C patient info:
final_pt_df2= pd.read_csv(os.path.join(new_data_path, 'nmh_icu_info.csv')) #anonymized patient and icustay info for NM-T and NM-C
#######NM tertiary data:

x_nmedw_train= pd.read_csv(os.path.join(new_data_path, 'x_tert_train.csv'))
y_nmedw_train= pd.read_csv(os.path.join(new_data_path, 'y_tert_train.csv'))
icu_nmedw_train= pd.read_csv(os.path.join(new_data_path, 'icustay_tert_train.csv'))

x_nmedw_test= pd.read_csv(os.path.join(new_data_path, 'x_tert_test.csv'))
y_nmedw_test= pd.read_csv(os.path.join(new_data_path, 'y_tert_test.csv'))
icu_nmedw_test= pd.read_csv(os.path.join(new_data_path, 'icustay_tert_test.csv'))


#######community data:
x_nmedw_com= pd.read_csv(os.path.join(new_data_path, 'x_com.csv'))
y_nmedw_com= pd.read_csv(os.path.join(new_data_path, 'y_com.csv'))
icu_nmedw_com= pd.read_csv(os.path.join(new_data_path, 'icustay_com.csv'))

####MIMIC data:
mimic_final_pt= pd.read_csv(os.path.join(new_data_path, 'mimic_icu_info.csv')) #anonymized patient and icustay info for MIMIC

x_mimic_test= pd.read_csv(os.path.join(new_data_path, 'x_mimic_test.csv'))
y_mimic_test= pd.read_csv(os.path.join(new_data_path, 'y_mimic_test.csv'))
icu_mimic_test= pd.read_csv(os.path.join(new_data_path, 'icu_mimic_test.csv'))

x_mimic_train= pd.read_csv(os.path.join(new_data_path, 'x_mimic_train.csv'))
y_mimic_train= pd.read_csv(os.path.join(new_data_path, 'y_mimic_train.csv'))
icu_mimic_train= pd.read_csv(os.path.join(new_data_path, 'icu_mimic_train.csv'))

## quick dataset distribution breakdown:

In [ ]:
x_nmedw_test.shape

In [ ]:
y_nmedw_train['0'].value_counts()

In [ ]:
y_nmedw_test['0'].value_counts()

In [ ]:
y_mimic_train['0'].value_counts()

In [ ]:
y_mimic_test['0'].value_counts()

In [ ]:
final_pt_df2.head(1)

In [ ]:
len(final_pt_df2)

In [ ]:
mimic_final_pt.head(1)

In [ ]:
len(mimic_final_pt)

# 5 experiments:
* Exp1: MIMIC trained model -> test on: MIMIC_test ; Recalibrate on Tertiary_train -> test on:NMEDW_Test
* Exp2: Tertiary trained model -> test on: Tertiary_test ; Recalibrate on MIMIC_train -> test on:MIMIC_test
* Exp3: Train on Pooled(NMEDW_train + MIMIC_train), test on MIMIC_test;  NMEDW_Test.
* Exp4: Ensemble (MIMIC,NM_tert), test on MIMIC_test;  NMEDW_Test
* Exp5: test all above models on NM Community 



# Exp1: MIMIC trained model -> test on: MIMIC_test ; Recalibrate on Tertiary_train -> test on:NMEDW_Test


In [ ]:
x=np.array(x_mimic_train.copy())
y=y_mimic_train.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()

z_subject_id= pd.merge(pd.DataFrame(icu_mimic_train), mimic_final_pt[['icustay_id','subject_id']], how='left')['subject_id'] #7205

In [ ]:
###rf: HYPERTUNING
# Number of trees in random forest
n_estimators = [25, 50, 150, 250]
# Number of features to consider at every split
max_features = [3,10,20,'auto']
# Maximum number of levels in tree
max_depth = [5, 7, 10, 15] 
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 5, 10]
# Method of selecting samples for training each tree. supposedly better with false when classes aren't perfectly ballanced
bootstrap = [True] 

param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

model= RandomForestClassifier(criterion='entropy', random_state=12345)

rf_hyper_exp_mim=hypertuning_fxn(x, y, nfolds=nfolds, model=model , param_grid=param_grid,z_subject_id=z_subject_id, scoring=scoring,n_iter = n_iter, gridsearch=False) #2.2 min with gridsearch false. 


In [ ]:
###model fitting and calibrating to training data
rf_mimic = CalibratedClassifierCV(rf_hyper_exp_mim.best_estimator_, method='sigmoid', cv=10)
rf_mimic.fit(x, y)

# hypertuned_cv_fxn(x, y, model_in, nfolds, lr_override=False)
rf_hyper_exp_mim2= hypertuned_cv_fxn(np.array(x_mimic_train.copy()), y, rf_mimic, nfolds=nfolds, z_subject_id=z_subject_id, lr_override=True)
rf_hyper_exp_mim2['tp_threshold'] #0.10776977266402202

### mimic model calibrated on mimic_train:


In [ ]:
mimic_model={'model':rf_mimic, 
            'threshold':rf_hyper_exp_mim2['tp_threshold'],
            'calibration':'mimic_train'}

### mimic model calibrated on Tertiary_train:

In [ ]:
### using the mimic target train data to find classification threshold
x=np.array(x_nmedw_train.copy())
y=y_nmedw_train.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()
z_subject_id= icu_nmedw_train



In [ ]:
#### recalibrating mimic model to nmedw train:

rf_mimic_recal = CalibratedClassifierCV(mimic_model['model'], method='sigmoid', cv="prefit")
rf_mimic_recal.fit(x, y)

rf_mimic_recal_cv= hypertuned_cv_fxn(x, y, rf_mimic_recal, nfolds=10, lr_override=False, z_subject_id=z_subject_id)

In [ ]:
mimic_model_recal={'model':rf_mimic_recal, 
            'threshold':rf_mimic_recal_cv['tp_threshold'],
            'calibration':'tert_train'}

### evaluate MIMIC models

In [ ]:

print('### MIMIC MODEL ON MIMIC TEST DATA')
rf_mimic_test= classifier_eval(rf_mimic, x=np.array(x_mimic_test), y=y_mimic_test,
                         training=False,train_threshold= rf_hyper_exp_mim2['tp_threshold'],
                         model_name='rf', folder_name=folder)


In [ ]:
print('MIMIC RF MODEL: NMEDW TEST')

x_test=np.array(x_nmedw_test.copy())
y_test=y_nmedw_test.copy() #copy of y_train
y_test=y_test.astype('int')
y_test=np.array(y_test).ravel()
z_subject_id= icu_nmedw_test

rf_mimic_eval_exp1= classifier_eval(mimic_model_recal['model'], x=np.array(x_test), y=y_test,
                              training=False,
                              train_threshold=mimic_model_recal['threshold'],
                              model_name='rf_mimic_recal', folder_name='24_hr_window')


In [ ]:
mimic_model.update({'results_mimicD-test':rf_mimic_test})
mimic_model_recal.update({'results_tertD-test':rf_mimic_eval_exp1})

In [ ]:
results_internal= mimic_model['results_mimicD-test'].copy()
results_internal['train_set']='MIMIC'
results_internal['test_set']='MIMIC'
results_internal['calibration']=mimic_model['calibration']

results_external= mimic_model_recal['results_tertD-test'].copy()
results_external['train_set']='MIMIC'
results_external['test_set']='tert'
results_external['calibration']=mimic_model_recal['calibration']

test_summary_exp1_df= pd.DataFrame([results_internal,results_external])

test_summary_exp1_df=test_summary_exp1_df.round(decimals=3).sort_values('auc', ascending=False)
test_summary_exp1_df=test_summary_exp1_df[['train_set','test_set','calibration','auc','f1','npv','precision','recall','threshold']]
display(test_summary_exp1_df)

In [ ]:
y_nmedw_train['0'].value_counts()/len(y_nmedw_train)

# Exp2: Tertiary trained model -> test on: Tertiary_test ; Recalibrate on MIMIC_train -> test on:MIMIC_test

In [ ]:
x=np.array(x_nmedw_train.copy())
y=y_nmedw_train.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()
z_subject_id= pd.merge(pd.DataFrame(icu_nmedw_train), final_pt_df2[['icustay_id','subject_id']], how='left')['icustay_id'] 

In [ ]:
##model hypertuning using same previously instantiated hyperparameter grid
model= RandomForestClassifier(criterion='entropy', random_state=12345)

rf_hyper_exp2=hypertuning_fxn(x, y, nfolds=nfolds, model=model , param_grid=param_grid,z_subject_id=z_subject_id, scoring=scoring,n_iter = n_iter, gridsearch=False)


In [ ]:
##model hypertuning using same previously instantiated hyperparameter grid
rf_tert = CalibratedClassifierCV(rf_hyper_exp2.best_estimator_, method='sigmoid', cv=10)
rf_tert.fit(x, y)

rf_tert_cv= hypertuned_cv_fxn(x, y, rf_tert, nfolds=nfolds, lr_override=True, z_subject_id=z_subject_id)

In [ ]:
##model dictionary w/ results, threshold and model object

tert_model={'model':rf_tert, 
            'threshold':rf_tert_cv['tp_threshold'],
            'calibration':'tert_train'}

In [ ]:
### recalibrating model to the respective training set for desired test set evaluation
x=np.array(x_mimic_train.copy())
y=y_mimic_train.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()
z_subject_id= pd.merge(pd.DataFrame(icu_mimic_train), mimic_final_pt[['icustay_id','subject_id']], how='left')['subject_id']


rf_tert_recal = CalibratedClassifierCV(tert_model['model'], method='sigmoid', cv="prefit")
rf_tert_recal.fit(x, y)

rf_tert_recal_cv= hypertuned_cv_fxn(x, y, rf_tert_recal, nfolds=10, lr_override=False, z_subject_id=z_subject_id)

In [ ]:
tert_model_recal={'model':rf_tert_recal, 
            'threshold':rf_tert_recal_cv['tp_threshold'],
            'calibration':'mimic_train'}

### evaluate Tertiary models

In [ ]:
print('Tertiary MODEL ON NMEDW TEST DATA')
tert_eval_tert= classifier_eval(tert_model['model'], x=np.array(x_nmedw_test), y=y_nmedw_test,
                         training=False,train_threshold= tert_model['threshold'],
                         model_name='rf', folder_name=folder)


In [ ]:
print('Tertiary MODEL ON MIMIC TEST DATA')
tert_eval_mimic_test= classifier_eval(tert_model_recal['model'], x=np.array(x_mimic_test), y=y_mimic_test,
                         training=False,train_threshold= tert_model_recal['threshold'],
                         model_name='rf_recal', folder_name=folder)


In [ ]:
tert_model.update({'results_tertD-test':tert_eval_tert})
tert_model_recal.update({'results_mimicD-test':tert_eval_mimic_test})

In [ ]:
results_internal= tert_model['results_tertD-test'].copy()
results_internal['train_set']='TERT'
results_internal['test_set']='TERT'
results_internal['calibration']=tert_model['calibration']

results_external= tert_model_recal['results_mimicD-test'].copy()
results_external['train_set']='TERT'
results_external['test_set']='MIMIC'
results_external['calibration']=tert_model_recal['calibration']


test_summary_exp2_df= pd.DataFrame([results_internal,results_external])

test_summary_exp2_df=test_summary_exp2_df.round(decimals=3).sort_values('auc', ascending=False)
test_summary_exp2_df=test_summary_exp2_df[['train_set','test_set','calibration','auc','f1','npv','precision','recall','threshold']]
display(test_summary_exp2_df)

# experiment 3: Exp3: Train on POOLED  (NMEDW_train + MIMIC_train), test on MIMIC_test ; NMEDW_Test

In [ ]:

mim=pd.merge(icu_mimic_test, mimic_final_pt[['icustay_id','final_bin']])
edw=pd.merge(icu_nmedw_test, final_pt_df2[['icustay_id','final_bin']])
mim['source']='mimic'
edw['source']='edw'



In [ ]:
y_mimic_train['0'].value_counts()

In [ ]:
y_nmedw_train['0'].value_counts()


In [ ]:
def merger(y_mimic_test, y_nmedw_test, x_nmedw_test,x_mimic_test, icu_mimic_test,icu_nmedw_test ):

    """
    function to sample an equal # of samples from NM-T and MIMIC, but to preserve each dataset's BI prevalence in the process. 
    """
    ##counts
    l_mim= len(y_mimic_test)
    l_nm= len(y_nmedw_test)
    display(l_mim, l_nm)

    min_n=min(l_mim, l_nm)

    if l_mim!=min_n:
        neg_n=c1=sum(y_mimic_test['0']==0)
        pos_n=c1=sum(y_mimic_test['0']==1)
        neg_ratio= neg_n/len(y_mimic_test)

        n_neg= round(neg_ratio*min_n)
        n_pos= min_n-n_neg

        ytest_1= y_mimic_test[y_mimic_test['0']==0].sample(n=n_neg, replace=False, random_state=12345)
        ytest_2= y_mimic_test[y_mimic_test['0']==1].sample(n=n_pos, replace=False, random_state=12345)
        y_pooled_test2= pd.concat([y_nmedw_test, ytest_1, ytest_2]) #[edw, mimic]


        mim2=pd.merge(icu_mimic_test.loc[pd.concat([ytest_1, ytest_2]).index], mimic_final_pt[['icustay_id','final_bin']])
        edw2=pd.merge(icu_nmedw_test, final_pt_df2[['icustay_id','final_bin']])
        
        mim2['icustay_id']=mim2['icustay_id']*100 ## doing this to make sure mimic and nmedw icustays don't overlap
         
        mim2['source']='mimic'
        edw2['source']='edw'
        icu_pooled_test2= pd.concat([edw2,mim2]) #[edw, mimic]

        x_pooled_test2= pd.concat([x_nmedw_test,x_mimic_test.loc[ytest_1.index], x_mimic_test.loc[ytest_2.index]]) #[edw, mimic]

        return(y_pooled_test2,x_pooled_test2,icu_pooled_test2 )

In [ ]:
y_pooled_test2,x_pooled_test2,icu_pooled_test2 = merger(y_mimic_test, y_nmedw_test, x_nmedw_test,x_mimic_test, icu_mimic_test,icu_nmedw_test )

## QC to ensure proper dimensions of pooled dataset
print(
len(y_pooled_test2),
len(x_pooled_test2),
len(icu_pooled_test2))
icu_pooled_test2['source'].value_counts()

In [ ]:
y_pooled_train2,x_pooled_train2,icu_pooled_train2 = merger(y_mimic_train, y_nmedw_train, x_nmedw_train,x_mimic_train, icu_mimic_train,icu_nmedw_train )
print(
len(y_pooled_train2),
len(x_pooled_train2),
len(icu_pooled_train2))
icu_pooled_train2['icustay_id']=icu_pooled_train2['icustay_id']#.astype('float')
icu_pooled_train2['source'].value_counts()


In [ ]:

x=np.array(x_pooled_train2.copy())
y=y_pooled_train2.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()
z_subject_id= icu_pooled_train2['icustay_id']


In [ ]:
print(icu_pooled_train2['icustay_id'].nunique())
print(icu_pooled_test2['icustay_id'].nunique())

In [ ]:
icu_pooled_train2['icustay_id'].nunique()

In [ ]:
##model hypertuning using same previously instantiated hyperparameter grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

model= RandomForestClassifier(criterion='entropy', random_state=12345)
rf_hyper_pooled=hypertuning_fxn(x, y, nfolds=nfolds, model=model , param_grid=param_grid, z_subject_id=z_subject_id, scoring=scoring,n_iter = n_iter, gridsearch=False)


In [ ]:
## adding this manually for the pooled hypertuning because the different anonomyzed icustay_id's for the demo are messing w/ the seed and yielding slightly different results
rf_hyper_pooled.best_estimator_.max_features=20
rf_hyper_pooled.best_estimator_.max_features

In [ ]:
rf_pooled = CalibratedClassifierCV(rf_hyper_pooled.best_estimator_, method='sigmoid', cv=10)

rf_pooled.fit(x, y)

rf_pooled_cv= hypertuned_cv_fxn(x, y, rf_pooled, nfolds=10, lr_override=True, z_subject_id=z_subject_id)



In [ ]:
pooled_model={'model':rf_pooled, 
            'threshold':rf_pooled_cv['tp_threshold'],
            'calibration':'pooled_train'}

### recalibrating pooled models to each dataset prior to testing on respective test data

In [ ]:
### recalibrating model to the respective training set for desired test set evaluation
x=np.array(x_mimic_train.copy())
y=y_mimic_train.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()
z_subject_id= pd.merge(pd.DataFrame(icu_mimic_train), mimic_final_pt[['icustay_id','subject_id']], how='left')['subject_id']

rf_pooled_recal_mim = CalibratedClassifierCV(pooled_model['model'], method='sigmoid', cv="prefit")
rf_pooled_recal_mim.fit(x, y)

rf_pooled_recal_mim_cv= hypertuned_cv_fxn(x, y, rf_pooled_recal_mim, nfolds=10, lr_override=True, z_subject_id=z_subject_id)

In [ ]:
##model dictionary w/ results, threshold and model object
pooled_model_recal_mim={'model':rf_pooled_recal_mim, 
            'threshold':rf_pooled_recal_mim_cv['tp_threshold'],
            'calibration':'mim_train'}

In [ ]:
### recalibrating model to the respective training set for desired test set evaluation
x=np.array(x_nmedw_train.copy())
y=y_nmedw_train.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()
z_subject_id= pd.merge(pd.DataFrame(icu_nmedw_train), final_pt_df2[['icustay_id','subject_id']], how='left')['icustay_id'] #7205


rf_pooled_recal_tert = CalibratedClassifierCV(pooled_model['model'], method='sigmoid', cv="prefit")
rf_pooled_recal_tert.fit(x, y)

rf_pooled_recal_tert_cv= hypertuned_cv_fxn(x, y, rf_pooled_recal_tert, nfolds=10, lr_override=True, z_subject_id=z_subject_id)

In [ ]:
##model dictionary w/ results, threshold and model object
pooled_model_recal_tert={'model':rf_pooled_recal_tert, 
            'threshold':rf_pooled_recal_tert_cv['tp_threshold'],
            'calibration':'tert_train'}

In [ ]:
### model evaluation: pooled model on mimic testset
pooled_mim_eval= classifier_eval(pooled_model_recal_mim['model'], x=np.array(x_mimic_test), y=y_mimic_test,
                         training=False,train_threshold= pooled_model_recal_mim['threshold'],
                         model_name='rf', folder_name=folder)

In [ ]:
### model evaluation: pooled model on NM-T testset
pooled_tert_eval= classifier_eval(pooled_model_recal_tert['model'], x=np.array(x_nmedw_test), y=y_nmedw_test,
                         training=False,train_threshold= pooled_model_recal_tert['threshold'],
                         model_name='rf', folder_name=folder)

In [ ]:
pooled_model_recal_mim.update({'results_mimicD-test':pooled_mim_eval})
pooled_model_recal_tert.update({'results_tertD-test':pooled_tert_eval})


In [ ]:
results_external_mim= pooled_model_recal_mim['results_mimicD-test'].copy()
results_external_mim['train_set']='POOLED'
results_external_mim['test_set']='MIMIC'
results_external_mim['calibration']=pooled_model_recal_mim['calibration']

results_external_tert= pooled_model_recal_tert['results_tertD-test'].copy()
results_external_tert['train_set']='POOLED'
results_external_tert['test_set']='TERT'
results_external_tert['calibration']=pooled_model_recal_tert['calibration']


test_summary_exp3_df= pd.DataFrame([results_external_mim, results_external_tert])#, logreg_mimic_test,logreg_mimic_eval_exp1])
test_summary_exp3_df=test_summary_exp3_df.round(decimals=3).sort_values('auc', ascending=False)
test_summary_exp3_df=test_summary_exp3_df[['train_set','test_set','calibration','auc','f1','npv','precision','recall','threshold']]
display(test_summary_exp3_df)

## Exp4: Ensembling Tertiary and MIMIC models
* recalibrate on respective training data before ensembling

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
#### ensembling mimic tailored NM-Tm and MIMICm:
#############fitting
x=np.array(x_mimic_train.copy())
y=y_mimic_train.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()
#time_interval=4
z_subject_id= pd.merge(pd.DataFrame(icu_mimic_train), mimic_final_pt[['icustay_id','subject_id']], how='left')['icustay_id'] 

clf_list_calibrated_mim = [tert_model_recal['model'], mimic_model['model']]

ensemble_mim_model = VotingClassifier(estimators = [('tert_recal' ,tert_model_recal['model']), ('mimic_cal', mimic_model['model'])], voting='soft')

ensemble_mim_model.estimators_ = clf_list_calibrated_mim
ensemble_mim_model.le_ = LabelEncoder().fit(y)
ensemble_mim_model.classes_ = ensemble_mim_model.le_.classes_

# Now it will work without calling fit
ensemble_cal_cv_mim= hypertuned_cv_fxn(x, y, ensemble_mim_model, nfolds=10, lr_override=True, z_subject_id=z_subject_id)


ensemble_mimic={'model':ensemble_mim_model, 
            'threshold':ensemble_cal_cv_mim['tp_threshold'],
            'calibration':'mimic_train'}

### rf: cat model on cat testset (balanced)
ensemble_eval_mim= classifier_eval(ensemble_mimic['model'], x=np.array(x_mimic_test), y=y_mimic_test,
                         training=False,train_threshold= ensemble_mimic['threshold'],#cv_cat_df.loc['RandomForestClassifier','tp_threshold'],
                         model_name='ensemble_cal_mimic', folder_name=folder)

In [ ]:
ensemble_cal_cv_mim

In [ ]:
ensemble_mimic.update({'results_mimicD-test':ensemble_eval_mim})

In [ ]:
#### ensembling NM-T tailored NM-Tm and MIMICm:
#############fitting
x=np.array(x_nmedw_train.copy())
y=y_nmedw_train.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()
#time_interval=4
z_subject_id= pd.merge(pd.DataFrame(icu_nmedw_train), final_pt_df2[['icustay_id','subject_id']], how='left')['icustay_id'] #7205


clf_list_calibrated_tert= [tert_model['model'], mimic_model_recal['model']]

ensemble_tert_model = VotingClassifier(estimators = [('tert_cal' ,tert_model['model']), ('mimic_recal', mimic_model_recal['model'])], voting='soft')

ensemble_tert_model.estimators_ = clf_list_calibrated_tert
ensemble_tert_model.le_ = LabelEncoder().fit(y)
ensemble_tert_model.classes_ = ensemble_tert_model.le_.classes_

# Now it will work without calling fit
ensemble_cal_cv_tert= hypertuned_cv_fxn(x, y, ensemble_tert_model, nfolds=10, lr_override=True, z_subject_id=z_subject_id)


ensemble_tert={'model':ensemble_tert_model, 
            'threshold':ensemble_cal_cv_tert['tp_threshold'],
            'calibration':'tert_train'}

### rf: cat model on cat testset (balanced)
ensemble_eval_tert= classifier_eval(ensemble_tert['model'], x=np.array(x_nmedw_test), y=y_nmedw_test,
                         training=False,train_threshold= ensemble_tert['threshold'],
                         model_name='ensemble_cal_tert', folder_name=folder)

ensemble_tert.update({'results_tertD-test':ensemble_eval_tert})

In [ ]:
##model dictionary w/ results, threshold and model object

results_internal= ensemble_tert['results_tertD-test'].copy()
results_internal['train_set']='ENSEMBLE'
results_internal['test_set']='TERT'
results_internal['calibration']=ensemble_tert['calibration']


results_external= ensemble_mimic['results_mimicD-test'].copy()
results_external['train_set']='ENSEMBLE'
results_external['test_set']='MIMIC'
results_external['calibration']=ensemble_mimic['calibration']

test_summary_exp4_df= pd.DataFrame([results_internal,results_external])

test_summary_exp4_df=test_summary_exp4_df.round(decimals=3).sort_values('auc', ascending=False)
test_summary_exp4_df=test_summary_exp4_df[['train_set','test_set','calibration','auc','f1','npv','precision','recall','threshold']]
display(test_summary_exp4_df)

# result compiling for exp 1-3

In [ ]:
model_dict_list=[mimic_model,
                 mimic_model_recal,

                tert_model,
                tert_model_recal,

                pooled_model,
                pooled_model_recal_mim,
                pooled_model_recal_tert,

                ensemble_mimic,
                ensemble_tert
                ]

In [ ]:
display(test_summary_exp1_df.reset_index())
display(test_summary_exp2_df.reset_index())
display(test_summary_exp3_df.reset_index())
display(test_summary_exp4_df)

# model saving

In [ ]:
def model_save1(model, model_name):
    import pickle
    modelpath=str(repository_path)+'/models/{}_{}'.format(date,'august_final')

    if not os.path.exists(modelpath):
        print(modelpath)
        os.makedirs(modelpath)

    filename = str(modelpath)+'/calibrated_{}.sav'.format(model_name)
    print(filename)
    pickle.dump(model, open(filename, 'wb'))
      
###code to save models:
# model_save1(mimic_model['model'],'rf_mimic_mim_cal')
# model_save1(mimic_model_recal['model'],'rf_mimic_tert_recal')

# model_save1(tert_model['model'],'rf_tert_tert_cal')
# model_save1(tert_model_recal['model'],'rf_tert_mim_recal')

# model_save1(pooled_model['model'],'rf_pooled_mer_cal')
# model_save1(pooled_model_recal_mim['model'],'rf_pooled_mim_recal')
# model_save1(pooled_model_recal_tert['model'],'rf_pooled_tert_recal')


# model_save1(ensemble_mimic['model'],'rf_ensemble_mim_cal')
# model_save1(ensemble_tert['model'],'rf_ensemble_tert_cal')



# applying models to community hospitals

In [ ]:
x=np.array(x_nmedw_com.copy())
y=y_nmedw_com.copy() #copy of y_train
y=y.astype('int')
y=np.array(y).ravel()
z_subject_id= icu_nmedw_com

In [ ]:
model_dict_list=[mimic_model,
                 mimic_model_recal,

                tert_model,
                tert_model_recal,

                pooled_model,
                pooled_model_recal_mim,
                pooled_model_recal_tert,

                ensemble_mimic,
                ensemble_tert
                ]
    
com_model_dict_list= [mimic_model_recal,
                      tert_model,
                      pooled_model_recal_tert,
                      ensemble_tert
    ]

In [ ]:
### model evaluation of each model (calibrated to NM-Ttrain) on NM-C
mimic_eval_com= classifier_eval(mimic_model_recal['model'], x=x, y=y,
                         training=False,train_threshold= mimic_model_recal['threshold'],#cv_cat_df.loc['RandomForestClassifier','tp_threshold'],
                         model_name='rf', folder_name=folder)

tert_eval_com= classifier_eval(tert_model['model'], x=x, y=y,
                         training=False,train_threshold= tert_model['threshold'],#cv_cat_df.loc['RandomForestClassifier','tp_threshold'],
                         model_name='rf', folder_name=folder)

pooled_eval_com= classifier_eval(pooled_model_recal_tert['model'], x=x, y=y,
                         training=False,train_threshold= pooled_model_recal_tert['threshold'],#cv_cat_df.loc['RandomForestClassifier','tp_threshold'],
                         model_name='rf', folder_name=folder)

ensemble_eval_com= classifier_eval(ensemble_tert['model'], x=x, y=y,
                         training=False,train_threshold= ensemble_tert['threshold'],#cv_cat_df.loc['RandomForestClassifier','tp_threshold'],
                         model_name='rf', folder_name=folder)



mimic_model_recal.update({'results_com':mimic_eval_com})
tert_model.update({'results_com':tert_eval_com})
pooled_model_recal_tert.update({'results_com':pooled_eval_com})
ensemble_tert.update({'results_com':ensemble_eval_com})


In [ ]:
# model dictionary for each nm-c result

com_model_dict_list= [mimic_model_recal,
                      tert_model,
                      pooled_model_recal_tert,
                      ensemble_tert
    ]
i=0
for element in com_model_dict_list:
    print(i)
    y_proba= element['model'].predict_proba(x_nmedw_com)[:,1]
    element.update({'com_proba':y_proba})
    element.update({'com_pred':[1 if y >= element['threshold'] else 0 for y in y_proba]})
    i+=1

In [ ]:
mimic_eval_com= classifier_eval(mimic_model_recal['model'], x=x, y=y,
                         training=False,train_threshold= mimic_model_recal['threshold'],
                         model_name='rf', folder_name=folder)

tert_eval_com= classifier_eval(tert_model['model'], x=x, y=y,
                         training=False,train_threshold= tert_model['threshold'],
                         model_name='rf', folder_name=folder)

pooled_eval_com= classifier_eval(pooled_model_recal_tert['model'], x=x, y=y,
                         training=False,train_threshold= pooled_model_recal_tert['threshold'],
                         model_name='rf', folder_name=folder)

ensemble_eval_com= classifier_eval(ensemble_tert['model'], x=x, y=y,
                         training=False,train_threshold= ensemble_tert['threshold'],
                         model_name='rf', folder_name=folder)


In [ ]:
### nm-C result table: 
tert_eval_com['train_set']='NMEDW_NMH'
tert_eval_com['test_set']='NMEDW_COMMUNITY'

mimic_eval_com['train_set']='MIMIC'
mimic_eval_com['test_set']='NMEDW_COMMUNITY'

pooled_eval_com['train_set']='POOLED'
pooled_eval_com['test_set']='NMEDW_COMMUNITY'

ensemble_eval_com['train_set']='ENSEMBLE'
ensemble_eval_com['test_set']='NMEDW_COMMUNITY'

test_com_summary_df= pd.DataFrame([mimic_eval_com,tert_eval_com,pooled_eval_com,ensemble_eval_com])

test_com_summary_df=test_com_summary_df.round(decimals=3)#.sort_values('auc', ascending=False)
test_com_summary_df['model']='RandomForestClassifier'

test_com_summary_df=test_com_summary_df[['train_set','test_set','auc','f1','npv','precision','recall','threshold']]

display(test_com_summary_df)

# calibration Evaluation (calcs and plots)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def cal_plot(X_test, y_test,clf_list, save_bool=False, plotname='calplot'):
    """
    calibration plot, taken from SKlearn documentation and modified slightly
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.calibration import CalibrationDisplay

    sns.set(font_scale=2.5)
    plt.style.use('seaborn-white')
    fig = plt.figure(figsize=(20, 20))
    gs = GridSpec(4, 2)
    
    from matplotlib.colors import ListedColormap
    colors = ListedColormap(sns.color_palette())

    ax_calibration_curve = fig.add_subplot(gs[:2, :2])
    calibration_displays = {}
    for i, (clf, name) in enumerate(clf_list):
        display = CalibrationDisplay.from_estimator(
            clf,
            X_test,
            y_test,
            n_bins=10,
            name=name+'$_{\mathrm{M}}$',
            ax=ax_calibration_curve,
            color=colors(i),
        )
        calibration_displays[name] = display
       
    ax_calibration_curve.grid()
    ax_calibration_curve.legend(bbox_to_anchor=(1.05, 0.45), loc='upper left', borderaxespad=0)

    # Add histogram
    grid_positions = [(2, 0), (2, 1), (3, 0), (3, 1)]
    for i, (_, name) in enumerate(clf_list):
        row, col = grid_positions[i]
        ax = fig.add_subplot(gs[row, col])

        ax.hist(
            calibration_displays[name].y_prob,
            range=(0, 1),
            bins=10,
            label=name,
            color=colors(i),
        )
        ax.set(title=name, xlabel="Mean predicted probability", ylabel="Count")

        
    plt.tight_layout()
    
    if save_bool==True:
        saveplot(plt, plotname, pubres=True)
    plt.show()
    

In [ ]:
save_boolean=False #change to True to save the plot

In [ ]:
com_model_dict_list= [mimic_model_recal,
                      tert_model,
                      pooled_model_recal_tert,
                      ensemble_tert
    ]

clf_list = [
    (mimic_model_recal['model'], "MIMIC"),
    (tert_model['model'], "NM-T"),
    (pooled_model_recal_tert['model'], "Pooled"),
    (ensemble_tert['model'], "Ensemble"),   
]

cal_plot(X_test= x_nmedw_com, y_test= y_nmedw_com, clf_list=clf_list, save_bool=save_boolean, plotname='nm_com_calplot_final')

In [ ]:
mimic_model_dict_list= [mimic_model,
                      tert_model_recal,
                      pooled_model_recal_mim,
                      ensemble_mimic
    ]

clf_list_mim = [
    (mimic_model['model'], "MIMIC"),
    (tert_model_recal['model'], "NM-T"),
    (pooled_model_recal_mim['model'], "Pooled"),
    (ensemble_mimic['model'], "Ensemble"),
    
]

cal_plot(X_test= x_mimic_test, y_test= y_mimic_test, clf_list=clf_list_mim, save_bool=save_boolean, plotname='mim_calplot_final')

In [ ]:
com_model_dict_list= [mimic_model_recal,
                      tert_model,
                      pooled_model_recal_tert,
                      ensemble_tert
    ]

clf_list = [
    (mimic_model_recal['model'], "MIMIC"),
    (tert_model['model'], "NM-T"),
    (pooled_model_recal_tert['model'], "Pooled"),
    (ensemble_tert['model'], "Ensemble"),
    
]

cal_plot(X_test= x_nmedw_test, y_test= y_nmedw_test, clf_list=clf_list, save_bool=save_boolean, plotname='nm_tert_calplot_final')

### Calibration Statistics

In [ ]:
from sklearn.metrics import brier_score_loss
def z_cal(oi,Ei):
    from scipy.stats import norm
    from math import sqrt
    """
    Spiegelhalter’s z-score neatly drops out the unavoidable penalty
    term by taking the difference of the score with the expectation.
    Schematically it is defined as

    
    
    If |Z(E, O)| > q1 − α/2⁠, where qα is the 
    α-quantile of the standard normal distribution (0.05), 
    the result is significant, suggesting an improperly calibrated model.
    """
    alpha = 0.05
    
    y_tert_z1= sum(
        (oi-Ei) * (1-(2*Ei))
    )
    y_tert_z2= sqrt(sum(
        ((1-(2*Ei))**2) * (Ei*(1-Ei))

    ))
    z_score=y_tert_z1/y_tert_z2
    if (abs(z_score) > norm.cdf(1-(alpha/2))):
        print('reject null. NOT calibrated')
    else:
        print('fail to reject. calibrated')
        
    print('z score: ', z_score, '\n')
    print('p value: ', 1-norm.cdf(abs(z_score)), '\n')
    return(z_score)


def calibration_calcs(model, dataset_x, dataset_y):
    from scipy.stats import linregress
    import math
   
    oi=dataset_y['0']
    Ei= model.predict_proba(dataset_x)[:,1]



#     print('Average absolute error: {}'.format(AAE(oi=oi,Ei=Ei)))
    print('Brier score: {}'.format(brier_score_loss(y_true=oi, y_prob=Ei)))
#     print(brier_score_loss(y_true=oi, y_prob=Ei))
    print('Spiegelhalter’s z-test:')
    print(z_cal(oi=oi,Ei=Ei))

    print(linregress(x=Ei, y=oi))

In [ ]:
clf_list = [
    (mimic_model_recal['model'], "mimic"),
    (tert_model['model'], "nm_tertiary"),
    (pooled_model_recal_tert['model'], "pooled_tertcal"),
    (ensemble_tert['model'], "ensemble")    
]

i=0
for element in clf_list:
    print('######### MODEL: {} #########'.format(clf_list[i][1]))
    calibration_calcs(clf_list[i][0], dataset_x=x_nmedw_com, dataset_y=y_nmedw_com)
    print()
    i+=1

### mean calibration

In [ ]:
m1=mimic_model['model'].predict_proba(x_mimic_test)[:,1]
m2=tert_model_recal['model'].predict_proba(x_mimic_test)[:,1]
m3=pooled_model_recal_mim['model'].predict_proba(x_mimic_test)[:,1]
m4=ensemble_mimic['model'].predict_proba(x_mimic_test)[:,1]

t1=mimic_model_recal['model'].predict_proba(x_nmedw_test)[:,1]
t2=tert_model['model'].predict_proba(x_nmedw_test)[:,1]
t3=pooled_model_recal_tert['model'].predict_proba(x_nmedw_test)[:,1]
t4=ensemble_tert['model'].predict_proba(x_nmedw_test)[:,1]

c1=mimic_model_recal['model'].predict_proba(x_nmedw_com)[:,1]
c2=tert_model['model'].predict_proba(x_nmedw_com)[:,1]
c3=pooled_model_recal_tert['model'].predict_proba(x_nmedw_com)[:,1]
c4=ensemble_tert['model'].predict_proba(x_nmedw_com)[:,1]


In [ ]:
m_mean_list=[]
m_std_list=[]
for element in [m1, m2, m3, m4]:
    m_mean_list.append(np.mean(element))
    m_std_list.append(np.std(element))
    
t_mean_list=[]
t_std_list=[]
for element in [t1, t2, t3, t4]:
    t_mean_list.append(np.mean(element))
    t_std_list.append(np.std(element))
    
c_mean_list=[]
c_std_list=[]
for element in [c1, c2, c3, c4]:
    c_mean_list.append(np.mean(element))
    c_std_list.append(np.std(element))
    
pd.DataFrame(index=['MIMICD', 'TertiaryD', 'NM-CommunityD'], )

In [ ]:
# pd.DataFrame(np.transpose([m_mean_list,t_mean_list,c_mean_list]))
display(pd.DataFrame(data=([m_mean_list,t_mean_list,c_mean_list]),
            index=['MIMICD', 'TertiaryD', 'NM-CommunityD'],
            columns=['MIMICm','Tertiarym','Pooledm','Ensemblem']).round(decimals=2))

display(pd.DataFrame(data=([m_std_list,t_std_list,c_std_list]),
            index=['MIMICD', 'TertiaryD', 'NM-CommunityD'],
            columns=['MIMICm','Tertiarym','Pooledm','Ensemblem']).round(decimals=2))

## stacked roc curve

In [ ]:

def find_neighbours(value, df, colname):
    exactmatch = df[df[colname] == value]
    if not exactmatch.empty:
        return exactmatch.index
    else:
        lowerneighbour_ind = df[df[colname] < value][colname].idxmax()
        upperneighbour_ind = df[df[colname] > value][colname].idxmin()
        return [lowerneighbour_ind, upperneighbour_ind]
    
    


def roc_publishing(model, x, y, proba_input=False,pos_label=1, print_default=True, model_name=None):
    import sklearn.metrics as metrics
    from sklearn.metrics import precision_score, roc_auc_score, f1_score, recall_score
    
    model_name=type(model).__name__

    y_proba = model.predict_proba(x)[:,1]
        
    fpr, tpr, thresholds = metrics.roc_curve(y, y_proba, pos_label=pos_label)
    roc_auc = metrics.auc(fpr, tpr)
    
    roc_df= pd.DataFrame({"thresholds": thresholds,"fpr":fpr, "tpr": tpr})
    roc_df.iloc[0,0] =1
    
    return(fpr, tpr, roc_auc, roc_df)



def stacked_roc(x_test, y_test, models_dic, first_bold=True, save_bool=False, plotname='stacked_roc'):
    """
    plotting function to plot a stacked ROC based on models in a dictionary. 
    first_bold=True means that the first model in the dic will stand out and be a solid line, while others are dotted
    """
    
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set(font_scale=2)
    plt.style.use('seaborn-white')
    plt.rcParams['figure.figsize'] = [16, 10]

    for model_name in models_dic.keys():

        model=models_dic[model_name]['model']
        fpr, tpr, roc_auc, roc_df= roc_publishing(model, x=np.array(x_test), y=y_test, model_name=model_name)
        print(model_name, roc_auc)
        ax1= plt.plot(fpr, tpr, label = '{}'.format(model_name)+'$_{\mathrm{M}}$'+ ' AUC = {:.3f}'.format(roc_auc), linestyle='solid', linewidth=3, alpha=0.75)

#         ##labeling the point w/ tuned high sensitivity threshold
#         idx=find_neighbours(value=models_dic[model_name]['threshold'], df=roc_df, colname='thresholds')[0]   
        
#         ### comment this out to remove the threshold values:
#         plt.plot(roc_df.iloc[idx,1], roc_df.iloc[idx,2],marker='o', markersize=8, color='black') ##
        
    ###annotating the plot
    plt.legend(loc = 'lower right')   

    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate', size=32)
    plt.xlabel('False Positive Rate', size=32)
    plt.grid(color='grey', linestyle='-', linewidth=1, alpha=0.2)

    if save_bool==True:
        saveplot(plt,plotname, pubres=True)
    else: pass
    plt.show()
    
    

In [ ]:
com_model_dict_list= [mimic_model_recal,
                      tert_model,
                      pooled_model_recal_tert,
                      ensemble_tert,
    ]

keys= ['MIMIC', 'NM-T', 'Pooled', 'Ensemble']
com_dic={}
for i in range(0,len(com_model_dict_list)):
    com_dic.update({keys[i] : com_model_dict_list[i]} )
    
    

In [ ]:
print('ROC for models on NMEDW Community Hospital Data')
stacked_roc(x_test=x_nmedw_com, y_test=y_nmedw_com, models_dic=com_dic, first_bold=False, save_bool=save_boolean, plotname='stacked_roc_com_final')

In [ ]:
mim_model_dict_list= [mimic_model,
                      tert_model_recal,
                      pooled_model_recal_mim,
                      ensemble_mimic,
    ]

keys= ['MIMIC', 'NM-T', 'Pooled', 'Ensemble']
mim_model_dic={}
for i in range(0,len(mim_model_dict_list)):
    mim_model_dic.update({keys[i] : mim_model_dict_list[i]} )

In [ ]:
print('ROC for models on MIMIC test Data')
stacked_roc(x_test=x_mimic_test, y_test=y_mimic_test, models_dic=mim_model_dic, first_bold=False, save_bool=save_boolean, plotname='stacked_roc_mim_final')

In [ ]:
stacked_roc(x_test=x_nmedw_test, y_test=y_nmedw_test, models_dic=com_dic, first_bold=False, save_bool=save_boolean, plotname='stacked_roc_tert_final')

# delong test for ROC comparisons

In [ ]:

import numpy as np
import scipy.stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.

    Args:
       x - a 1D numpy array
    Returns:
       array of midranks

    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2



def compute_midrank_weight(x, sample_weight):
    """Computes midranks.

    Args:
       x - a 1D numpy array
    Returns:
       array of midranks

    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2



def fastDeLong(predictions_sorted_transposed, label_1_count):
    """Fast DeLong test computation.

    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }

    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov



def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.

    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)

    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)



def compute_ground_truth_statistics(ground_truth, sample_weight=None):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight



def delong_roc_variance(ground_truth, predictions):
    """Computes ROC AUC variance for a single set of predictions.

    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1

    """
    sample_weight = None
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov



def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """Computes log(p-value) for hypothesis that two ROC AUCs are different.
    ** modified to return non-logscaled p-value

    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1

    """
    order, label_1_count, _ = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return 10**calc_pvalue(aucs, delongcov)



In [ ]:
print('##### NMEDW Tertiary: #####')
print('mimic v tertiary: {}'.format(
    delong_roc_test(y_nmedw_test['0'].values, mimic_model_recal['model'].predict_proba(x_nmedw_test)[:,1], tert_model['model'].predict_proba(x_nmedw_test)[:,1])[0][0])
)

print('mimic v pooled: {}'.format(
    delong_roc_test(y_nmedw_test['0'].values, pooled_model_recal_tert['model'].predict_proba(x_nmedw_test)[:,1], mimic_model_recal['model'].predict_proba(x_nmedw_test)[:,1])[0][0])
     )
print('mimic v ensemble: {}'.format(
    delong_roc_test(y_nmedw_test['0'].values, ensemble_tert['model'].predict_proba(x_nmedw_test)[:,1], mimic_model_recal['model'].predict_proba(x_nmedw_test)[:,1])[0][0])
     )

print('tertiary v pooled: {}'.format(
    delong_roc_test(y_nmedw_test['0'].values, pooled_model_recal_tert['model'].predict_proba(x_nmedw_test)[:,1], tert_model['model'].predict_proba(x_nmedw_test)[:,1])[0][0])
     )
print('tertiary v ensemble: {}'.format(
    delong_roc_test(y_nmedw_test['0'].values, tert_model['model'].predict_proba(x_nmedw_test)[:,1], ensemble_tert['model'].predict_proba(x_nmedw_test)[:,1])[0][0])
     )


print('pooled v ensemble: {}'.format(
    delong_roc_test(y_nmedw_test['0'].values, pooled_model_recal_tert['model'].predict_proba(x_nmedw_test)[:,1], ensemble_tert['model'].predict_proba(x_nmedw_test)[:,1])[0][0])
)


In [ ]:
print('##### MIMIC: #####')
print('mimic v tertiary: {}'.format(
    delong_roc_test(y_mimic_test['0'].values, mimic_model['model'].predict_proba(x_mimic_test)[:,1], tert_model_recal['model'].predict_proba(x_mimic_test)[:,1])[0][0])
)

print('mimic v pooled: {}'.format(
    delong_roc_test(y_mimic_test['0'].values, pooled_model_recal_mim['model'].predict_proba(x_mimic_test)[:,1], mimic_model['model'].predict_proba(x_mimic_test)[:,1])[0][0])
     )
print('mimic v ensemble: {}'.format(
    delong_roc_test(y_mimic_test['0'].values, ensemble_mimic['model'].predict_proba(x_mimic_test)[:,1], mimic_model['model'].predict_proba(x_mimic_test)[:,1])[0][0])
     )

print('tertiary v pooled: {}'.format(
    delong_roc_test(y_mimic_test['0'].values, pooled_model_recal_mim['model'].predict_proba(x_mimic_test)[:,1], tert_model_recal['model'].predict_proba(x_mimic_test)[:,1])[0][0])
     )
print('tertiary v ensemble: {}'.format(
    delong_roc_test(y_mimic_test['0'].values, tert_model_recal['model'].predict_proba(x_mimic_test)[:,1], ensemble_mimic['model'].predict_proba(x_mimic_test)[:,1])[0][0])
     )


print('pooled v ensemble: {}'.format(
    delong_roc_test(y_mimic_test['0'].values, pooled_model_recal_mim['model'].predict_proba(x_mimic_test)[:,1], ensemble_mimic['model'].predict_proba(x_mimic_test)[:,1])[0][0])
)

In [ ]:
print('##### NMEDW Community: #####')
print('mimic v tertiary: {}'.format(
    delong_roc_test(y_nmedw_com['0'].values, mimic_model_recal['com_proba'], tert_model['com_proba'])[0][0])
)

print('mimic v pooled: {}'.format(
    delong_roc_test(y_nmedw_com['0'].values, pooled_model_recal_tert['com_proba'], mimic_model_recal['com_proba'])[0][0])
     )
print('mimic v ensemble: {}'.format(
    delong_roc_test(y_nmedw_com['0'].values, mimic_model_recal['com_proba'], ensemble_tert['com_proba'])[0][0])
     )

print('tertiary v pooled: {}'.format(
    delong_roc_test(y_nmedw_com['0'].values, pooled_model_recal_tert['com_proba'], tert_model['com_proba'])[0][0])
     )
print('tertiary v ensemble: {}'.format(
    delong_roc_test(y_nmedw_com['0'].values, tert_model['com_proba'], ensemble_tert['com_proba'])[0][0])
     )


print('pooled v ensemble: {}'.format(
    delong_roc_test(y_nmedw_com['0'].values, pooled_model_recal_tert['com_proba'], ensemble_tert['com_proba'])[0][0])
)

# Precision vs. Recall plot

In [ ]:

def roc_publishing(model, x, y, proba_input=False,pos_label=1, print_default=True, model_name=None):
    import sklearn.metrics as metrics
    from sklearn.metrics import precision_score, roc_auc_score, f1_score, recall_score

    model_name=type(model).__name__

    y_proba = model.predict_proba(x)[:,1]
        
    fpr, tpr, thresholds = metrics.roc_curve(y, y_proba, pos_label=pos_label)
    roc_auc = metrics.auc(fpr, tpr)
    
    #gathering the optimal youden_index and df of tpr/fpr for auc and index of that optimal youden. idx is needed in the roc
    youden_threshold, roc_df, idx= optimal_youden_index(fpr, tpr, thresholds, tp90=True)
    
    return(fpr, tpr, roc_auc, roc_df, idx)



def pr_publishing(model_dic, x, y, pos_label=1, print_default=True, model_name=None):
    import sklearn.metrics as metrics
    from sklearn.metrics import precision_recall_curve

    model=model_dic['model']
    
    model_name=type(model).__name__

    y_proba = model.predict_proba(x)[:,1]
    y_pred=[1 if y >= model_dic['threshold'] else 0 for y in y_proba]
    
    lr_precision, lr_recall, thresholds = metrics.precision_recall_curve(y, y_proba, pos_label=pos_label)
    lr_f1 = f1_score(y, y_pred)
    lr_auc = auc(lr_recall, lr_precision)

    return(lr_precision, lr_recall,thresholds, lr_f1, lr_auc)


    
def stacked_pr(x_test, y_test, models_dic, save_bool=False, plotname='stacked_pr'):
    """
    plotting function to plot a stacked ROC based on models in a dictionary. 
    first_bold=True means that the first model in the dic will stand out and be a solid line, while others are dotted
    """
    
    import matplotlib.pyplot as plt
    import seaborn as sns
    
#     global save_boolean
    sns.set(font_scale=2)
    plt.style.use('seaborn-white')
    plt.rcParams['figure.figsize'] = [16, 10]
    
    for model_name in models_dic.keys():
        model=models_dic[model_name]['model']
        precision, recall, thresholds, f1, auc= pr_publishing(models_dic[model_name], x=np.array(x_test), y=y_test, model_name=model_name)
        print(model_name, auc, f1)
        ax1= plt.plot(precision, recall, label = '{}'.format(model_name)+'$_{\mathrm{M}}$'+ ' AUC = {:.3f}   F1 = {:.3f}'.format(auc, f1), linestyle='solid', linewidth=3, alpha=0.75)
        
        ###annotating the plot
    plt.legend(loc = 'lower left')  

    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('Precision', size=32)
    plt.xlabel('Recall', size=32)
    plt.grid(color='grey', linestyle='-', linewidth=1, alpha=0.2)
    
    if save_bool==True:
        saveplot(plt,plotname, pubres=True)
    else: pass
    plt.show()
    


In [ ]:
com_model_dict_list= [mimic_model_recal,
                      tert_model,
                      pooled_model_recal_tert,
                      ensemble_tert
    ]

keys= ['MIMIC', 'NM-T', 'Pooled', 'Ensemble']
com_dic={}
for i in range(0,len(com_model_dict_list)):
    com_dic.update({keys[i] : com_model_dict_list[i]} )
    
    

In [ ]:
stacked_pr(x_test=x_nmedw_com, y_test=y_nmedw_com['0'], models_dic=com_dic, save_bool=save_boolean, plotname='stacked_pr_com_final')


In [ ]:
stacked_pr(x_test=x_mimic_test, y_test=y_mimic_test['0'], models_dic=mim_model_dic, save_bool=save_boolean, plotname='stacked_pr_mim_final')


In [ ]:
stacked_pr(x_test=x_nmedw_test, y_test=y_nmedw_test['0'], models_dic=com_dic, save_bool=save_boolean, plotname='stacked_pr_tert_final')


# variable importance exploration

In [ ]:
#### uses permutation based importances for all now
def common_imp_variables(x_train, y_train, models_dic):
    """
    function that takes in a dictionary of models and the x_train dataframe and returns the set of variables present in the combined list of each model's top N most important variables.
    said another way, takes each model's top N variables (via permutation importance) and returns the set of all variables. 
    1) find top N variables for each model
    2) make list of all models top N
    3) filter to only unique values in list = imp_var_names
    
    note: The purpose of this function is to run the permutation variable importance w/ the entire predictor space only twice, then once the top important variables across models are found (next function), can run permutation again focused only on them.
    this saves significant computational time.
    """
    global n_varimp
    features_dic={}
    top_set_dic={}

    for model_name in models_dic.keys():
        model= models_dic[model_name]
        print(model_name)
        
        from sklearn.inspection import permutation_importance
        result = permutation_importance(
            model, x_train.values, y_train, n_repeats=2, random_state=12345, n_jobs=-1
        )

        feature_names= list(x_train)
        forest_importances = pd.Series(result.importances_mean, index=feature_names)
        features=forest_importances.nlargest(n_varimp).sort_values()
        features=list(features.reset_index()['index'])
        features_dic.update( {model_name :features } )

    #######
    set_features=[]

    for features in features_dic.values():
        set_features=set_features+features
    set_features=set(set_features)
    imp_var_names=list(set_features)

    return(imp_var_names)


#### uses permutation based importances for all now

def perm_impvar_calc(models_dic, imp_var_names, x_train, y_train):
    """
    calculates the relative variable importance of the set of each model's topN important variables on the supplied x and y data. 
    input:
        models_dic:dictionary of models 
        imp_var_names: the top N set of important variables among all models
        x_train: desired predictor matrix to permute importance on
        y_train: desired predictor matrix to permute importance on
    output: 
        relative variable importance for each model of all set(imp_var_names) variables.
    note: relative variable importance determined by dividing each variable importance by the value of the most important variable. this makes all values a comparison to the most important varaible:
    ie 50 rel variable importance = half as important as the most important variable
    """
    
    # finding the index of the set(varimp_names) in the dataframe.  
    #getting index of the top variables in the predictor matrix. 
    xtrain_column_index_list=[]
    for element in imp_var_names:
        variable_index=list(x_train).index(element)
        xtrain_column_index_list.append(variable_index)
    
    rel_result_dic={} #instantiating dictionary
    result_dic={}
    for model_name in models_dic.keys():
        model= models_dic[model_name]
        print(model_name)           
        from sklearn.inspection import permutation_importance
        result = permutation_importance(
            model, x_train.values, y_train, n_repeats=10, random_state=12345, n_jobs=-1
        )

        feature_names= list(x_train)
        imp = pd.Series(result.importances_mean, index=feature_names)[imp_var_names]
        imp=imp.sort_values()
        rel_imp=100.0 * (imp / imp.max())
        
        features =list(np.array(x_train.columns)[xtrain_column_index_list])
        top_set= rel_imp
        rel_result_dic.update( {model_name :top_set } ) #scaled to be relative
        result_dic.update( {model_name :result } ) #not scaled to be relative

    return(rel_result_dic, result_dic)


def perm_impvar_plot(result_set_dic_p, imp_var_names,models_dic, xvar_rotation=80, sort_base='nm_tertiary', figsize=[8,5], save_bool=False, plotname='perm_varimp'):
    """
    input: 
        result_set_dic_p: permuted variable importance dictionary from topN_rel_imp2(),
        imp_var_names: top important variables to plot,
        models_dic: ,
        xvar_rotation: rotation of the xvariable label on the plot
        sort_base: name of the model to provide sorting basis.
        figsize: dimensions for figure output,
        save_bool: boolean to save plot using saveplot function
        plotname: name of plot used in saveplot function. 
    output:
        importance_df: dataframe of calculated relative variable importances for additional plotting 
        error_df: same but w/ calculated permutation stdev (scaled to the ratio of value:error prior to making importances relative. )
   
    """
    from sklearn.inspection import permutation_importance
    feature_names= list(x_nmedw_train)
    
    SMALL_SIZE = 8+4
    MEDIUM_SIZE = 10+4
    BIGGER_SIZE = 12+4

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    plt.rcParams['figure.figsize'] = figsize 
    
    imp_dict={}
    err_dict={}

    for model_name in models_dic.keys(): ##now that we have set of top N variables for each model. we can make relative importance for all unique variables in the set
        
        model= models_dic[model_name]
        print(model_name)
        
        result= result_set_dic_p[model_name]
        
        forest_importances = pd.Series(result.importances_mean, index=feature_names)[varimp_names]
        forest_errors=  pd.Series(result.importances_std, index=feature_names)[varimp_names]
        
        ###sorting
        forest_importances=forest_importances.sort_values()
        forest_errors=forest_errors[forest_importances.index]
        
        # scaing valuse to be relative values
        rel_imp=100.0 * (forest_importances / forest_importances.max())
        # scaling errors to be the same ratio of value:error pre-scaling
        forest_errors_scale= forest_errors/forest_importances
        rel_imp_errors= forest_errors_scale * rel_imp
        
        imp_dict.update({model_name :rel_imp })
        err_dict.update({model_name :rel_imp_errors })
        
        
    importance_df=pd.DataFrame(imp_dict)
    error_df=pd.DataFrame(err_dict)

    importance_df=abs(importance_df.sort_values(sort_base))

    fig, ax = plt.subplots()
    importance_df.plot.bar(yerr=error_df.loc[importance_df.index], ax=ax, capsize=4, rot=80)
    
    if save_bool==True:
        saveplot(plt,plotname)
       
    plt.show()
    return(importance_df,error_df)

In [ ]:
def perm_impvar_plot_prep(result_set_dic_p, imp_var_names,models_dic, sort_base='Ensemble'):
    """
    input: 
        result_set_dic_p: permuted variable importance dictionary from topN_rel_imp2(),
        imp_var_names: top important variables to plot,
        models_dic: ,
        sort_base: name of the model to provide sorting basis.
    output:
        importance_df: dataframe of calculated relative variable importances for additional plotting 
        error_df: same but w/ calculated permutation stdev (scaled to the ratio of value:error prior to making importances relative. )
   
    """
    from sklearn.inspection import permutation_importance
    feature_names= list(x_nmedw_train)
    
    imp_dict={}
    err_dict={}

    for model_name in models_dic.keys(): ##now that we have set of top N variables for each model. we can make relative importance for all unique variables in the set
        model= models_dic[model_name]
        print(model_name)
        
        result= result_set_dic_p[model_name]
        
        forest_importances = pd.Series(result.importances_mean, index=feature_names)[imp_var_names]
        forest_errors=  pd.Series(result.importances_std, index=feature_names)[imp_var_names]
        
        ###sorting
        forest_importances=forest_importances.sort_values()
        forest_errors=forest_errors[forest_importances.index]
        
        # scaing valuse to be relative values
        rel_imp=100.0 * (forest_importances / forest_importances.max())
        # scaling errors to be the same ratio of value:error pre-scaling
        forest_errors_scale= forest_errors/forest_importances
        rel_imp_errors= forest_errors_scale * rel_imp
        
        imp_dict.update({model_name :rel_imp })
        err_dict.update({model_name :rel_imp_errors })
        
    importance_df=pd.DataFrame(imp_dict)
    error_df=pd.DataFrame(err_dict)

    importance_df=abs(importance_df.sort_values(sort_base))
    return(importance_df,error_df)

In [ ]:
def perm_impvar_plot(importance_df, error_df, x_var_labels, sort_base='Ensemble', xvar_rotation=80, figsize=[8,5], save_bool=False, plotname='perm_varimp'):
    """
    input: 
        importance_df: dataframe of calculated relative variable importances for additional plotting 
        error_df: same but w/ calculated permutation stdev (scaled to the ratio of value:error prior to making importances relative.
        xvar_rotation: rotation of the xvariable label on the plot
        figsize: dimensions for figure output,
        save_bool: boolean to save plot using saveplot function
        plotname: name of plot used in saveplot function. 

   
    """
    from sklearn.inspection import permutation_importance
    feature_names= list(x_nmedw_train)
    
    SMALL_SIZE = 8+4
    MEDIUM_SIZE = 10+4
    BIGGER_SIZE = 12+4

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    plt.rcParams['figure.figsize'] = figsize 
    


    importance_df=importance_df.sort_values('Ensemble', ascending=False).iloc[0:10,:]
    fig, ax = plt.subplots()

    importance_df.plot.bar(yerr=error_df.loc[importance_df.index], ax=ax, capsize=4, rot=xvar_rotation)
    plt.grid(color='grey', linestyle='-', linewidth=1, alpha=0.2)
    labels = [item.get_text() for item in ax.get_xticklabels()]

    for i in range(len(x_var_labels)):
        labels[i] = x_var_labels[i]

    ax.set_xticklabels(labels)
    ax.set_ylim([0, 120])
    ax.set_ylabel("Relative Variable Importance", size=25)

    if save_bool==True:
        saveplot(plt,plotname)
    
    plt.show()



In [ ]:
### setting up a model dictionary list in the expected format for the relative variable plotting functions. 
com_model_dict_list= [mimic_model_recal['model'],
                      tert_model['model'],
                      pooled_model_recal_tert['model'],
                      ensemble_tert['model']
    ]

keys= ['MIMIC', 'Tertiary', 'Pooled', 'Ensemble']
com_dic={}
for i in range(0,len(com_model_dict_list)):
    com_dic.update({keys[i] : com_model_dict_list[i]} )

## warning this code takes 30-60 min to run since it is permuting n=10 replicates on a model that is already in a double nested ensemble (due to the recalibrationCV). 

In [ ]:
# Permuting variable importances for all 4 models on Tertiary_train. 
n_varimp=10

#find set(topN) variables
imp_var_names= common_imp_variables(x_train=x_nmedw_train, y_train=y_nmedw_train, models_dic=com_dic) ##n varimp names not important for the df

#find rel importance of set(topN) variables for each model
rel_perm_imp_var_result, raw_perm_imp_var_result= perm_impvar_calc(models_dic=com_dic, imp_var_names=imp_var_names, x_train=x_nmedw_train, y_train=y_nmedw_train)


In [ ]:
#find rel importance of set(topN) variables for each model
# rel_perm_imp_var_result, raw_perm_imp_var_result= perm_impvar_calc(models_dic=com_dic, imp_var_names=imp_var_names, x_train=x_nmedw_train, y_train=y_nmedw_train)


In [ ]:
importance_df, error_df= perm_impvar_plot_prep(raw_perm_imp_var_result, imp_var_names, models_dic=com_dic, sort_base='Ensemble')

In [ ]:
importance_df.sort_values('Ensemble', ascending=False).iloc[0:12,:]


In [ ]:
importance_df2=importance_df.drop(index=['minWBC','bicarbonate','bilirubin','mingcs','resp'])

In [ ]:
importance_df2.sort_values('Ensemble', ascending=False).iloc[0:10,:]


In [ ]:
x_var_label_list=[
                'Temperature (max)',
                 'Blood Culture (+)',
                 'Leukocyte (+)',
                 'BUN (max)',
                 'Heartrate (max)',
                 'WBC (max)',
                 'PaO2:FiO2 (min)',
                 'Norepinephrine (+)',
                 'Respirate (max)',
                 'SBP (min)'
                ] #list of  cleaned up variable label names for publication

In [ ]:
###published varimp plot. 
perm_impvar_plot(importance_df2, error_df, x_var_labels=x_var_label_list, xvar_rotation=80, figsize=[20,10], save_bool=False, plotname='perm_varimp')


# membership model and assessing similarity between development and validation cohorts
https://www.sciencedirect.com/science/article/pii/S0895435614002753?via%3Dihub
. We hereto estimate a binary logistic regression model, further referred to as membership model, to predict the probability that an individual belongs to (is a member of) the development sample as compared with the validation sample. Hence, the dependent variable of this model is “1” for participants of the development set and “0” for those of the validation set. This model should at least include as independent variables the predictors and outcome from the original prediction model to ensure that model performance can (at least partially) be interpreted in terms of its considered predictors and outcome. It may be clear that if the membership model discriminates poorly (or well), both samples are strongly (or not much) related in terms of the considered predictor variables and outcome status

In [ ]:
y_nmedw_train_m= y_nmedw_train.copy()
y_nmedw_train_m['0']=1

y_mimic_train_m=y_mimic_train.copy()
y_mimic_train_m['0']=1

y_nmedw_com_m= y_nmedw_com.copy()
y_nmedw_com_m['0']=0

# y_mimic_train_m=y_mimic_train.copy()
# y_mimic_train_m['0']=1

In [ ]:
x_m_1= pd.concat([x_nmedw_train, x_nmedw_com]) 

x_m_2= pd.concat([x_mimic_train, x_nmedw_com]) 

y_m_1= pd.concat([y_nmedw_train_m, y_nmedw_com_m])

y_m_2= pd.concat([y_mimic_train_m, y_nmedw_com_m])

In [ ]:
x_m_1= pd.concat([x_nmedw_train, x_nmedw_com]) 

x_m_2= pd.concat([x_mimic_train, x_nmedw_com]) 

y_m_1= pd.concat([y_nmedw_train_m, y_nmedw_com_m])

y_m_2= pd.concat([y_mimic_train_m, y_nmedw_com_m])

lr_model_m_1= LogisticRegression(random_state=12345,solver='liblinear', penalty='l1')
lr_model_m_1.fit(x_m_1,y_m_1)

lr_model_m_2= LogisticRegression(random_state=12345,solver='liblinear', penalty='l1')
lr_model_m_2.fit(x_m_2,y_m_2)

In [ ]:
### rf: cat model on cat testset (balanced)
m_1_eval= classifier_eval(lr_model_m_1, x=np.array(x_m_1), y=y_m_1,
                         training=False,train_threshold= 0.5,
                         model_name='m', folder_name=folder)

In [ ]:
### rf: cat model on cat testset (balanced)
m_2_eval= classifier_eval(lr_model_m_2, x=np.array(x_m_2), y=y_m_2,
                         training=False,train_threshold= 0.5,
                         model_name='m', folder_name=folder)

In [ ]:
#This model should at least include as independent variables the predictors and outcome from the original prediction model to ensure that model performance can (at least partially) be interpreted in terms of its considered predictors and outcome. 
## it hink this means i should include my y variable as a predictor?

x_nmedw_train_m=x_nmedw_train.copy()
x_nmedw_train_m['y']=y_nmedw_train['0']

x_mimic_train_m=x_mimic_train.copy()
x_mimic_train_m['y']=y_mimic_train['0']

x_nmedw_com_m=x_nmedw_com.copy()
x_nmedw_com_m['y']=y_nmedw_com['0']


x_m_1= pd.concat([x_nmedw_train_m, x_nmedw_com_m]) 

x_m_2= pd.concat([x_mimic_train_m, x_nmedw_com_m]) 

y_m_1= pd.concat([y_nmedw_train_m, y_nmedw_com_m])

y_m_2= pd.concat([y_mimic_train_m, y_nmedw_com_m])

lr_model_m_1= LogisticRegression(random_state=12345,solver='liblinear', penalty='l2')
lr_model_m_1.fit(x_m_1,y_m_1)

lr_model_m_2= LogisticRegression(random_state=12345,solver='liblinear', penalty='l2')
lr_model_m_2.fit(x_m_2,y_m_2)

In [ ]:
x_pooled_train_m=x_pooled_train2.copy()
x_pooled_train_m['y']=y_pooled_train2['0']

y_pooled_train_m= y_pooled_train2.copy()
y_pooled_train_m['0']=1

x_m_3= pd.concat([x_pooled_train_m, x_nmedw_com_m]) 
y_m_3= pd.concat([y_pooled_train_m, y_nmedw_com_m])


lr_model_m_3= LogisticRegression(random_state=12345,solver='liblinear', penalty='l2')
lr_model_m_3.fit(x_m_3,y_m_3)

In [ ]:
# y_nmedw_train_m= y_nmedw_train.copy()
# y_nmedw_train_m['0']=1

y_mimic_train_m2=y_mimic_train.copy()
y_mimic_train_m2['0']=0


x_m_4= pd.concat([x_nmedw_train_m, x_mimic_train_m]) 
y_m_4= pd.concat([y_nmedw_train_m, y_mimic_train_m2])


lr_model_m_4= LogisticRegression(random_state=12345,solver='liblinear', penalty='l2')
lr_model_m_4.fit(x_m_4,y_m_4)

In [ ]:
### rf: cat model on cat testset (balanced)
m_1_eval= classifier_eval(lr_model_m_1, x=np.array(x_m_1), y=y_m_1,
                         training=False,train_threshold= 0.5,
                         model_name='m', folder_name=folder)

In [ ]:
### rf: cat model on cat testset (balanced)
m_2_eval= classifier_eval(lr_model_m_2, x=np.array(x_m_2), y=y_m_2,
                         training=False,train_threshold= 0.5,
                         model_name='m', folder_name=folder)

In [ ]:
### rf: cat model on cat testset (balanced)
m_3_eval= classifier_eval(lr_model_m_3, x=np.array(x_m_3), y=y_m_3,
                         training=False,train_threshold= 0.5,
                         model_name='m', folder_name=folder)

In [ ]:
### development model between MIMIC and NM-T
m_4_eval= classifier_eval(lr_model_m_4, x=np.array(x_m_4), y=y_m_4,
                         training=False,train_threshold= 0.5,
                         model_name='m', folder_name=folder)

## doing the same thing between NM-T train vs test and MIMIC train vs test

In [ ]:

x_nmedw_train_m=x_nmedw_train.copy()
x_nmedw_train_m['y']=y_nmedw_train['0']

x_nmedw_test_m=x_nmedw_test.copy()
x_nmedw_test_m['y']=y_nmedw_test['0']


y_nmedw_train_m= y_nmedw_train.copy()
y_nmedw_train_m['0']=1

y_nmedw_test_m= y_nmedw_test.copy()
y_nmedw_test_m['0']=0



x_mimic_train_m=x_mimic_train.copy()
x_mimic_train_m['y']=y_mimic_train['0']

x_mimic_test_m=x_mimic_test.copy()
x_mimic_test_m['y']=y_mimic_test['0']


y_mimic_train_m= y_mimic_train.copy()
y_mimic_train_m['0']=1

y_mimic_test_m= y_mimic_test.copy()
y_mimic_test_m['0']=0



x_m_11= pd.concat([x_nmedw_train_m, x_nmedw_test_m]) 

x_m_22= pd.concat([x_mimic_train_m, x_mimic_test_m]) 

y_m_11= pd.concat([y_nmedw_train_m, y_nmedw_test_m])

y_m_22= pd.concat([y_mimic_train_m, y_mimic_test_m])

lr_model_m_11= LogisticRegression(random_state=12345,solver='liblinear', penalty='l2')
lr_model_m_11.fit(x_m_11,y_m_11)

lr_model_m_22= LogisticRegression(random_state=12345,solver='liblinear', penalty='l2')
lr_model_m_22.fit(x_m_22,y_m_22)

In [ ]:
### development model between MIMIC and NM-T
m_11_eval= classifier_eval(lr_model_m_11, x=np.array(x_m_11), y=y_m_11,
                         training=False,train_threshold= 0.5,
                         model_name='m', folder_name=folder)

In [ ]:
### development model between MIMIC and NM-T
m_22_eval= classifier_eval(lr_model_m_22, x=np.array(x_m_22), y=y_m_22,
                         training=False,train_threshold= 0.5,
                         model_name='m', folder_name=folder)

## membership model variable importance

In [ ]:
#### the membership models to be explored for variable importance:
# ### rf: membership model between M:nm-t and nm-C. 0.82
# m_1_eval= classifier_eval(lr_model_m_1, x=np.array(x_m_1), y=y_m_1,
#                          training=False,train_threshold= 0.5,
#                          model_name='m', folder_name=folder)

# ### rf:  membership model between MIMIC and nm-C. 0.98
# m_2_eval= classifier_eval(lr_model_m_2, x=np.array(x_m_2), y=y_m_2,
#                          training=False,train_threshold= 0.5,
#                          model_name='m', folder_name=folder)

# ### rf:  membership model between POOLED and nm-C. 0.98
# m_3_eval= classifier_eval(lr_model_m_3, x=np.array(x_m_3), y=y_m_3,
#                          training=False,train_threshold= 0.5,
#                          model_name='m', folder_name=folder)


In [ ]:

def memb_importance(lr_model,x_m, y_m):
    from sklearn.inspection import permutation_importance
    model_fi = permutation_importance(lr_model, x_m, y_m, n_repeats=20, random_state=12345, n_jobs=-1)

    model_fi1=pd.DataFrame(data=model_fi['importances_mean'], columns=['rel_imp'])#, scoring= accuracy by default. aka the average decrease in accuracy when variable is permutted.  
    model_fi1['colname']= lr_model.feature_names_in_
    model_fi1['rel_imp2']=abs(model_fi1['rel_imp'])
    model_fi1=model_fi1.drop(columns='rel_imp').rename(columns={'rel_imp2':'imp'})
    model_fi1['imp_std']=model_fi['importances_std']
    model_fi1=model_fi1.sort_values('imp', ascending=False)#.reset_index(drop=True)
    model_fi1['rel_imp']= 100*(model_fi1['imp']/model_fi1['imp'].max())
    return(model_fi1)

def memb_importance_auc(lr_model,x_m, y_m):
    from sklearn.inspection import permutation_importance
    model_fi = permutation_importance(lr_model, x_m, y_m,  scoring=['roc_auc'], n_repeats=20, random_state=12345, n_jobs=-1)

    model_fi1=pd.DataFrame(data=model_fi['roc_auc']['importances_mean'], columns=['rel_imp'])#, columns= 
    model_fi1['colname']= lr_model.feature_names_in_
    model_fi1['rel_imp2']=abs(model_fi1['rel_imp'])
    model_fi1=model_fi1.drop(columns='rel_imp').rename(columns={'rel_imp2':'imp'})
    model_fi1['imp_std']=model_fi['roc_auc']['importances_std']
    model_fi1=model_fi1.sort_values('imp', ascending=False)#.reset_index(drop=True)
    model_fi1['rel_imp']= 100*(model_fi1['imp']/model_fi1['imp'].max())
    return(model_fi1)

In [ ]:
##importance = average decrease in ACCURACY across n=10 permutations
nmt_nmc_imp=memb_importance(lr_model_m_1, x_m_1, y_m_1)
mim_nmc_imp=memb_importance(lr_model_m_2, x_m_2, y_m_2)
pool_nmc_imp=memb_importance(lr_model_m_3, x_m_3, y_m_3)

In [ ]:
##importance = average decrease in AUROC across n=10 permutations
nmt_nmc_imp2=memb_importance_auc(lr_model_m_1, x_m_1, y_m_1)
mim_nmc_imp2=memb_importance_auc(lr_model_m_2, x_m_2, y_m_2)
pool_nmc_imp2=memb_importance_auc(lr_model_m_3, x_m_3, y_m_3)

In [ ]:
memb_impvar_names=list(set(nmt_nmc_imp.head(6)['colname'].to_list()+ mim_nmc_imp.head(6)['colname'].to_list()+ pool_nmc_imp.head(6)['colname'].to_list()))

In [ ]:
###making relative errors
nmt_nmc_imp['rel_std']= nmt_nmc_imp['rel_imp']*(nmt_nmc_imp['imp_std']/nmt_nmc_imp['imp'])
mim_nmc_imp['rel_std']= mim_nmc_imp['rel_imp']*(mim_nmc_imp['imp_std']/mim_nmc_imp['imp'])
pool_nmc_imp['rel_std']= pool_nmc_imp['rel_imp']*(pool_nmc_imp['imp_std']/pool_nmc_imp['imp'])


nmt_nmc_imp_prep= nmt_nmc_imp.loc[nmt_nmc_imp['colname'].isin(memb_impvar_names),['colname','rel_imp','rel_std']]
mim_nmc_imp_prep= mim_nmc_imp.loc[mim_nmc_imp['colname'].isin(memb_impvar_names),['colname','rel_imp','rel_std']]
pool_nmc_imp_prep= pool_nmc_imp.loc[pool_nmc_imp['colname'].isin(memb_impvar_names),['colname','rel_imp','rel_std']]

In [ ]:
member_dic={
    'Tertiary':nmt_nmc_imp_prep.set_index('colname')['rel_imp'],
    'MIMIC':mim_nmc_imp_prep.set_index('colname')['rel_imp'],
    'Pooled':pool_nmc_imp_prep.set_index('colname')['rel_imp'],
}

member_error_dic={
    'Tertiary':nmt_nmc_imp_prep.set_index('colname')['rel_std'],
    'MIMIC':mim_nmc_imp_prep.set_index('colname')['rel_std'],
    'Pooled':pool_nmc_imp_prep.set_index('colname')['rel_std'],
}


In [ ]:
imp_df=pd.DataFrame(member_dic).sort_values('Tertiary', ascending=False)#.reset_index()
err_df= pd.DataFrame(member_error_dic)

In [ ]:
from sklearn.inspection import permutation_importance
import seaborn as sns
from matplotlib.colors import ListedColormap
colors =sns.color_palette()[0:3]
my_cmap = ListedColormap(sns.color_palette())#.as_hex())

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-white')
colors=ListedColormap(sns.color_palette('deep')).colors

xvar_rotation=80
figsize=[8,5]

# feature_names= list(x_nmedw_train)
x_var_labels=list(imp_df.index)

SMALL_SIZE = 8+4
MEDIUM_SIZE = 10+4
BIGGER_SIZE = 12+4

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams['figure.figsize'] = figsize 

imp_df=imp_df.sort_values('Tertiary', ascending=False).iloc[0:11,:]
imp_df = imp_df.rename_axis(None) #making sure the axis column title is removed
x_var_labels=list(imp_df.index)

fig, ax = plt.subplots()
imp_df.plot.bar(yerr=err_df.loc[imp_df.index], ax=ax, capsize=4, rot=xvar_rotation, color= [colors[0],colors[1],colors[2]])#, c=colors)

plt.grid(color='grey', linestyle='-', linewidth=1, alpha=0.2)
labels = [item.get_text() for item in ax.get_xticklabels()]

for i in range(len(x_var_label_list)):
    labels[i] = x_var_label_list[i]

ax.set_xticklabels(labels)
ax.set_ylim([0, 120])
ax.set_ylabel("Relative Variable Importance", size=15)

saveplot(plt,'membership_importance')

# modeling result heatmap

In [ ]:
display(test_summary_exp1_df.reset_index())
display(test_summary_exp2_df.reset_index())
display(test_summary_exp3_df.reset_index())
display(test_summary_exp4_df)
display(test_com_summary_df)

In [ ]:
com_model_dict_list= [mimic_model_recal,
                      tert_model,
                      pooled_model_recal_tert,
                      ensemble_tert,
    ]

keys= ['MIMIC', 'NM-T', 'Pooled', 'Ensemble']
com_dic={}
for i in range(0,len(com_model_dict_list)):
    com_dic.update({keys[i] : com_model_dict_list[i]} )
    

In [ ]:
hm=pd.DataFrame({'MIMIC_m':[0.782, 0.722,0.741],
                  'NM-T_m':[0.694,0.810,0.807],
                   'Pooled_m':[0.774,0.788, 0.795],
                  'Ensemble_m':[0.767,0.798, 0.798 ]
                  }
            ).transpose()#,     
hm=hm.rename(columns={0:'MIMICtest',1:'NM-Ttest',2:'NM-Cval'})

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(font_scale=2.5)
plt.style.use('seaborn-white')
plt.rcParams['figure.figsize'] = [12, 8]   

x_axis_labels=[]
y_axis_labels=[]

for element in ['MIMIC', 'NM-T', 'Pooled', 'Ensemble']:
    y_axis_labels.append('{}'.format(element)+'$_{\mathrm{M}}$')
    
for element in ['MIMIC', 'NM-T']:
    x_axis_labels.append('{}'.format(element)+'$_{\mathrm{test}}$')
x_axis_labels.append('{}'.format('NM-C')+'$_{\mathrm{val}}$')

ax = sns.heatmap(hm, annot=True,xticklabels=x_axis_labels, 
                 yticklabels=y_axis_labels,
                 cmap=sns.cubehelix_palette(start=.2, rot=-.3, as_cmap=True,dark=0.25, light=.75, reverse=True))
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

ax.xaxis.tick_top()
plt.tick_params(left = False, bottom = False, top=False)

# saveplot(plt,'auc_heatmap_4')

# sensitivity analysis

In [ ]:
comln_tert_model = label_bi_risk(x_com= x_nmedw_com_longneg,
                y_com= y_nmedw_com_longneg, icu_com= icu_nmedw_com_longneg, 
                model_in=tert_model , model_threshold= 0.2671307538155534, 
                pt_df=final_pt_df2)
comln_mim_model = label_bi_risk(x_com= x_nmedw_com_longneg,
                y_com= y_nmedw_com_longneg, icu_com= icu_nmedw_com_longneg, 
                model_in=mim_model , model_threshold= 0.274, 
                pt_df=final_pt_df2)

comln_ensemble_model = label_bi_risk(x_com= x_nmedw_com_longneg,
                y_com= y_nmedw_com_longneg, icu_com= icu_nmedw_com_longneg, 
                model_in=ensemble_tert_model , model_threshold= 0.267, 
                pt_df=final_pt_df2)

In [ ]:
### plotting function displayed in supplement
sns.set(rc={'figure.figsize':(10,8)}, font_scale = 1.3)
sns.set_style("white")
colors=sns.color_palette('deep').as_hex()

sns.distplot(comln_mim_model['y_proba_model'], hist=False, kde=True, color=colors[0],norm_hist=False, hist_kws=dict(alpha=1),label='MIMIC'+'$_{\mathrm{M}}$')

sns.distplot(comln_tert_model['y_proba_model'], hist=False, kde=True, color=colors[1],norm_hist=False, hist_kws=dict(alpha=0.5),label='NM-T'+'$_{\mathrm{M}}$')

sns.distplot(comln_ensemble_model['y_proba_model'], hist=False, kde=True, color=colors[2],norm_hist=False, hist_kws=dict(alpha=0.5),label='Ensemble'+'$_{\mathrm{M}}$')

plt.axvline(x=0.274, color='black',  linestyle='dashed', label='Prediction Threshold')
plt.legend(['MIMIC'+'$_{\mathrm{M}}$','NM-T'+'$_{\mathrm{M}}$','Ensemble'+'$_{\mathrm{M}}$','Prediction Threshold'])
plt.xlabel('Predicted Probability')
plt.ylabel('Density')
plt.show()


display(comln_mim_model['y_hat_model'].value_counts())
display(comln_tert_model['y_hat_model'].value_counts())
display(comln_ensemble_model['y_hat_model'].value_counts())

# Fairness analysis

In [ ]:
## bringing some fairness stats in:
black_bool=x_nmedw_com['ethnicity_black']==1
hispanic_bool= x_nmedw_com['ethnicity_hispanic']==1
other_bool= x_nmedw_com['ethnicity_unknown/other']==1
female_bool= x_nmedw_com['gender_M']==0


x=np.array(x_nmedw_com.copy())
y=y_nmedw_com.copy() #copy of y_train

In [ ]:
#### modified classifier_eval function to output additional information useful for fairness analysis

def classifier_eval(model,
                    x,
                    y,
                    proba_input=False,
                    pos_label=1,
                    training=True,
                    train_threshold=None,
                    print_default=True,
                    model_name=None,
                    folder_name=None,
                    save=save_boolean):
    import sklearn.metrics as metrics
    from sklearn.metrics import precision_score, roc_auc_score, f1_score, recall_score
    """
    classification evaluation function. able to print/save the following:
    
    print/save the following:
        ROC curve marked with threshold for optimal youden (maximizing tpr+fpr with constraint that tpr>0.9)

        using 0.5 threshold:
            confusion matrix
            classification report
            npv
            accuracy

        using optimal youden (maximizing tpr+fpr with constraint that tpr>0.9):
            confusion matrix
            classification report
            npv
            accuracy
    
    output: 
        outputs modelname, auc, precision, recall, f1, and npv to a dictionary. 
    
    notes:
    youden's J statistic:
    J= sensitivity + specificity -1
    (truepos/ truepos+falseneg) + (true neg/ trueneg + falsepos) -1. 
    NOTE: with tpr>0.9 turned on, the youden statistic is basically just the furthest point on the line away from the midline with tpr>=0.9
    NOTE2: this function arguably does too much. in the future it may be better to seperate it out into more compartmental functions like with preprocessing().
    """
    
    if proba_input==True: 
        y_proba= model
        y_pred=[1 if y >= 0.5 else 0 for y in y_proba]
    
    else:
        model_name=type(model).__name__

        y_pred = model.predict(x)
        y_proba = model.predict_proba(x)[:,1]
        
    if training==True:
        
        fpr, tpr, thresholds = metrics.roc_curve(y, y_proba, pos_label=pos_label)
        roc_auc = metrics.auc(fpr, tpr)

        #gathering the optimal youden_index and df of tpr/fpr for auc and index of that optimal youden. idx is needed in the roc
        tp_threshold, roc_df, idx= optimal_youden_index(fpr, tpr, thresholds,tp90=True)
    
    else: #if training is not true, then we use the tuned threshold specified on the trainingset.
        fpr, tpr, thresholds = metrics.roc_curve(y, y_proba, pos_label=pos_label)
        roc_auc = metrics.auc(fpr, tpr)
        roc_df= pd.DataFrame({"thresholds": thresholds,"fpr":fpr, "tpr": tpr})
        roc_df.iloc[0,0] =1
        tp_threshold= train_threshold

    #plotting roc
    #plot_roc(fpr, tpr, roc_auc, threshold, save=save_boolean,model_name=None, folder_name=None, file_name=None
    plot_roc(fpr, tpr, roc_auc, thresholds, tp_threshold, save=save_boolean, model_name=model_name,folder_name=folder)
    plt.show(), plt.close()
    
    #printing npv, recall, precision, accuracy
    npv=confusion_matrix(y, y_pred)[0,0]/sum(np.array(y_pred)==0)
    prec= precision_score(y_true=y, y_pred= y_pred, pos_label=pos_label)
    recall= recall_score(y_true=y, y_pred= y_pred, pos_label=pos_label)
    f1= f1_score(y_true=y, y_pred= y_pred, pos_label=pos_label)
    confusion =pd.DataFrame(confusion_matrix(y, y_pred),
                                 index=['condition_neg','condition_pos'],
                                columns=['pred_neg','pred_pos'])
    
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    specificity = tn / (tn+fp)
    
    if save==True:
        save_df(confusion, df_name='{}_confusion_base'.format(model_name), rel_path='/tables/', verbose=False)
    
    if print_default==True: ###can opt to not print the 0.5 classification threshold classification report/conf matrix
        #plotting confusion matrixs
        print("\n******* Using 0.5 Classification Threshold *******\n")
        print(confusion)
        print('\n')
        print ('the Accuracy is: {:01.3f}'.format(accuracy_score(y, y_pred)))
        print ("npv: {:01.3f}".format(npv))
        print ('the classification_report:\n', classification_report(y,y_pred, digits=3))
    else:
        pass
    
    #### YOUDEN ADJUSTMENT #####

    print("\n******* Using Optimal TPR>=0.9 Classification Threshold *******\n")
    print("\nthe Youden optimal index is : {:01.3f}".format(train_threshold))

    y_pred_youden = [1 if y >= train_threshold else 0 for y in y_proba]

    npv_y=confusion_matrix(y, y_pred_youden)[0,0]/sum(np.array(y_pred_youden)==0)
    prec_y= precision_score(y_true=y, y_pred= y_pred_youden, pos_label=pos_label)
    recall_y= recall_score(y_true=y, y_pred= y_pred_youden, pos_label=pos_label)
    f1_y= f1_score(y_true=y, y_pred= y_pred_youden, pos_label=pos_label)
    auc_y=roc_auc_score(y_true=y, y_score= y_proba)
    
    
    ##plotting and saving confusion matrix
    confusion_youden=pd.DataFrame(confusion_matrix(y, y_pred_youden),
                                 index=['condition_neg','condition_pos'],
                                columns=['pred_neg','pred_pos'])
    
    if save==True:
        save_df(confusion_youden, df_name='{}_confusion_tuned'.format(model_name), rel_path='/tables/',verbose=False)
    
    #plotting confusion matrixs
    print('\n')
    print(confusion_youden)
    print('\n')
    print ('the Accuracy is: {:01.3f}'.format(accuracy_score(y, y_pred_youden)))
    print ("npv: {:01.3f}".format(npv_y))
    print ('the classification_report:\n', classification_report(y,y_pred_youden, digits=3))
    
    youden_dic= {'model':model_name, 'auc':auc_y, 'precision':prec_y, 'recall':recall_y, 'f1':f1_y, 'npv':npv_y,'threshold':tp_threshold}
    return(youden_dic, confusion_youden)

In [ ]:
def fairness_analysis(group_bool):

    x_interest_group= x_nmedw_com[group_bool]
    y_interest_group= y_nmedw_com[group_bool]

    x_opinterest_group=x_nmedw_com[~group_bool]
    y_opinterest_group= y_nmedw_com[~group_bool]
    
    
    print('### TERT MODEL ON NM-C interest group')
    rf_tert_group_interest,rf_tert_group_interest2 = classifier_eval(tert_model, x=np.array(x_interest_group), y=y_interest_group,
                             training=False,train_threshold= 0.267,
                             model_name='rf', folder_name=folder, save=False, print_default=False)
    
    rf_tert_group_interest['model']='NM-T'
    rf_tert_group_interest['group']='interest'
    
    print('### TERT MODEL ON NM-C opinterest group')
    rf_tert_group_opinterest,rf_tert_group_opinterest2= classifier_eval(tert_model, x=np.array(x_opinterest_group), y=y_opinterest_group,
                             training=False,train_threshold= 0.267,
                             model_name='rf', folder_name=folder, save=False, print_default=False)
    rf_tert_group_opinterest['model']='NM-T'
    rf_tert_group_opinterest['group']='opinterest'
    
    print('### MIMIC MODEL ON NM-C interest group')
    rf_mimic_group_interest,rf_mimic_group_interest2= classifier_eval(mim_model, x=np.array(x_interest_group), y=y_interest_group,
                             training=False,train_threshold= 0.274,
                             model_name='rf', folder_name=folder, save=False, print_default=False)
    rf_mimic_group_interest['model']='MIMIC'
    rf_mimic_group_interest['group']='interest'
    
    print('### MIMIC MODEL ON NM-C opinterest group')
    rf_mimic_group_opinterest,rf_mimic_group_opinterest2= classifier_eval(mim_model, x=np.array(x_opinterest_group), y=y_opinterest_group,
                             training=False,train_threshold= 0.274,
                             model_name='rf', folder_name=folder, save=False, print_default=False)
    rf_mimic_group_opinterest['model']='MIMIC'
    rf_mimic_group_opinterest['group']='opinterest'
        
    summary_df= pd.DataFrame([rf_tert_group_interest, rf_tert_group_opinterest, rf_mimic_group_interest, rf_mimic_group_opinterest])
    summary_df=summary_df.round(decimals=3)#.sort_values('auc', ascending=False)
    group_results= {'tert_interest':rf_tert_group_interest2, 'tert_opinterest':rf_tert_group_opinterest2, 'mim_interest':rf_mimic_group_interest2, 'mim_opinterest':rf_mimic_group_opinterest2}
    summary_df=summary_df[['model','group','auc','f1','npv','precision','recall','threshold']]

    return(summary_df, group_results)

In [ ]:

def fairness_analysis_manual(results, bool_name):
    """
    function to manually calculate statistics related to fiarness
    """

    d1=results['tert_interest'] #underpriv
    d2=results['tert_opinterest'] #priv
    n_female=sum(d1.sum())
    n_male=sum(d2.sum())
    n_tot=len(bool_name)
    
    print('baseline disadvantaged prevalence: {:.3f}'.format(n_female/n_tot))
    print('baseline disadvantaged condition pos prev: {:.3f}'.format(d1.loc['condition_pos'].sum()/n_female))
    print('baseline advantaged condition pos prev: {:.3f}'.format(d2.loc['condition_pos'].sum()/n_male))
    
    # correct # w/ adjusted decision threshold

    print('######### TERTm ###########')
    
    p1z1= d1.loc[:,'pred_pos'].sum()/n_female #predicted pos given the sensitive group = True
    p1z0= d2.loc[:,'pred_pos'].sum()/n_male #predicted pos given the sensitive group = False

    p1z1y1= d1.loc['condition_pos','pred_pos'].sum()/n_female
    p1z0y1= d2.loc['condition_pos','pred_pos'].sum()/n_male
    
    p1=min((p1z1/p1z0),(p1z0/p1z1))
    eod1=min((p1z1y1/p1z0y1),(p1z0y1/p1z1y1))
    
    TPR1=  d1.loc['condition_pos','pred_pos']/ d1.loc['condition_pos'].sum()
    TPR2=  d2.loc['condition_pos','pred_pos']/ d2.loc['condition_pos'].sum()
    
    print('p% score: {:.3f}'.format(p1))

    print('equality of opportunity: {:.3f}'.format(eod1))
    print('equality of opportunity diff: {:.3f}'.format(TPR2-TPR1))
    print((p1z1y1/p1z0y1), (p1z0y1/p1z1y1))
    
    print('######### MIMICm ###########')
    # correct # w/ adjusted decision threshold
    d1=results['mim_interest']
    d2=results['mim_opinterest']

    n_female=sum(d1.sum())
    n_male=sum(d2.sum())
    n_tot=len(bool_name)


    p1z1= d1.loc[:,'pred_pos'].sum()/n_female #predicted pos given the sensitive group = True
    p1z0= d2.loc[:,'pred_pos'].sum()/n_male #predicted pos given the sensitive group = False

    p1z1y1= d1.loc['condition_pos','pred_pos'].sum()/n_female
    p1z0y1= d2.loc['condition_pos','pred_pos'].sum()/n_male

    p2=min((p1z1/p1z0),(p1z0/p1z1))
    eod2=min((p1z1y1/p1z0y1),(p1z0y1/p1z1y1))
          
    TPR1=  d1.loc['condition_pos','pred_pos']/ d1.loc['condition_pos'].sum()
    TPR2=  d2.loc['condition_pos','pred_pos']/ d2.loc['condition_pos'].sum()
    
    print('p% score: {:.3f}'.format(p2))

    print('equality of opportunity: {:.3f}'.format(eod2))
    print('equality of opportunity diff: {:.3f}'.format(TPR2-TPR1))
    print((p1z1y1/p1z0y1), (p1z0y1/p1z1y1))

In [ ]:
fairness_analysis_manual(black__results, black_bool)

# baseline disadvantaged prevalence: 0.061
# baseline disadvantaged condition pos prev: 0.464
# baseline advantaged condition pos prev: 0.445
# ######### TERTm ###########
# p% score: 0.876
# equality of opportunity: 0.916
# equality of opportunity diff: 0.112
# 0.9158486045903264 1.0918835220012328
# ######### MIMICm ###########
# p% score: 0.985
# equality of opportunity: 0.986
# equality of opportunity diff: 0.049

In [ ]:
fairness_analysis_manual(hispanic_results, hispanic_bool)
# baseline disadvantaged prevalence: 0.067
# baseline disadvantaged condition pos prev: 0.392
# baseline advantaged condition pos prev: 0.450
# ######### TERTm ###########
# p% score: 0.918
# equality of opportunity: 0.890
# equality of opportunity diff: -0.021
# 0.889522599793747 1.1241985310231233
# ######### MIMICm ###########
# p% score: 0.923
# equality of opportunity: 0.845
# equality of opportunity diff: 0.026

In [ ]:
fairness_analysis_manual(other_results, other_bool)
# baseline disadvantaged prevalence: 0.036
# baseline disadvantaged condition pos prev: 0.483
# baseline advantaged condition pos prev: 0.445
# ######### TERTm ###########
# p% score: 0.893
# equality of opportunity: 0.864
# equality of opportunity diff: -0.060
# 1.1573148712522858 0.8640690833929565
# ######### MIMICm ###########
# p% score: 0.996
# equality of opportunity: 0.931
# equality of opportunity diff: 0.010

In [ ]:
fairness_analysis_manual(female_results, female_bool)
# baseline disadvantaged prevalence: 0.499
# baseline disadvantaged condition pos prev: 0.446
# baseline advantaged condition pos prev: 0.446
# ######### TERTm ###########
# p% score: 0.986
# equality of opportunity: 0.993
# equality of opportunity diff: 0.007
# 0.9926281728444803 1.0074265745796787
# ######### MIMICm ###########
# p% score: 0.924
# equality of opportunity: 0.928
# equality of opportunity diff: 0.069